In [ ]:
import pandas as pd
import numpy as np
import re

In [ ]:
# mount drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df_ai = pd.read_csv('/content/drive/MyDrive/UChicago/NLP/topic_modeling_completed.csv')

In [ ]:
df_ai.shape

(181339, 8)

In [ ]:
df_ai.head()

,Unnamed: 0,url,date,title_clean,text_clean,topic,industry,Representation
0,0,http://businessnewsthisweek.com/business/infog...,2023-05-20,Infogain AI Business Solutions Now Available i...,Infogain AI Business Solutions Now Available i...,53,Business & Startups,"['small', 'entrepreneurs', 'unlocking', 'ango'..."
1,1,https://allafrica.com/stories/202504250184.html,2025-04-25,Africa AI Policies in Africa - Lessons From Gh...,Africa AI Policies in Africa - Lessons From Gh...,93,Employment & Workforce,"['biontech', 'intellitek', 'founded', 'streaml..."
2,2,https://asiatimes.com/2023/07/yang-lan-intervi...,2023-07-25,Yang Lan interviews academics on AI development,a link to a friend (Opens in new window)Click ...,261,Technology,"['office', 'ordered', 'holiday', 'highlighted'..."
3,3,https://cdn.meritalk.com/articles/commerce-nom...,2025-02-04,Commerce Nominee Promises Increased Domestic A...,"Details By Andrew Rice Feb 3, 2025 4 14 pm Com...",52,Government & Policy,"['biden', 'order', 'executive', 'safeguards', ..."
4,4,https://citylife.capetown/hmn/uncategorized/re...,2023-11-11,Revolutionizing the Manufacturing Industry The...,Revolutionizing the Manufacturing Industry The...,5,Employment & Workforce,"['usd', 'cagr', 'growth', 'projected', 'techna..."


In [ ]:
df_ai.drop(columns=['Unnamed: 0'], inplace=True)

## GPT label parts of data

In [ ]:
import openai
import os

In [ ]:
client = openai.OpenAI(api_key="YOUR_API_KEY_HERE")

def gpt_label(text):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "system",
                "content": "You are an assistant that classifies whether this text reflects a 'positive' or 'negative' sentiment about AI integration. Reply with only one word: 'positive' or 'negative'."
            },
            {
                "role": "user",
                "content": text[:2000]
            }
        ],
        temperature=0.0,
        max_tokens=10
    )
    return response.choices[0].message.content.strip().lower()

In [ ]:
df_ai['industry'].unique()

array(['Business & Startups', 'Employment & Workforce', 'Technology',
       'Government & Policy', 'Marketing & Sales', 'Healthcare',
       'Automotive', 'Finance & Stock Markets', 'Education',
       'Lifestyle & Family', 'Media & Entertainment', 'Legal Services',
       'Energy & Environment', 'Cybersecurity', 'Food & Restaurants'],
      dtype=object)

In [ ]:
# Sample usage
i=0
for industry in df_ai['industry'].unique():
  if industry == 'Technology': sample = df_ai[df_ai['industry'] == industry].sample(20)
  else: sample = df_ai[df_ai['industry'] == industry].sample(10)
  if i==0:
    sample_data = sample
    i=1
  else:
    sample_data = pd.concat([sample_data, sample])
sample_data.shape

(160, 7)

In [ ]:
sample_data.head()

,url,date,title_clean,text_clean,topic,industry,Representation
53363,https://www.cleveland19.com/prnewswire/2022/05/17/jina-ai-shapes-future-search-cb-insights-names-it-100-most-innovative-ai-startups-second-year-running/,2022-05-17,Jina AI Shapes Future of Search as CB Insights Names it in 100 Most Innovative AI Startups for Second Year Running,"Watch LiveVaccineBig Bad B MovieSeen OnContestsCMSD on CW43DealsSee It, Snap It, Send ItNews19 News Investigative UnitUnidentifiedScam SquadCrime400 Years The Vestiges of Slavery in ClevelandCOVID...",55,Business & Startups,"['list', 'startup', 'lab', 'fero', 'diveplane', 'indian', 'zenerate', 'encora', 'blackbaud', 'invests']"
30121,https://www.sakshipost.com/news/musk-confused-how-non-profit-openai-became-30-bn-profit-making-firm-170561,2023-03-16,Musk confused how non-profit OpenAI became 30 bn profit-making firm,News Andhra Pradesh Telangana National International Technology Business Social News Brief Crime Covid 4th Wave Politics Andhra Pradesh Politics Telangana Politics National Politics International ...,10,Business & Startups,"['truthgpt', 'lawsuit', 'profit', 'altman', 'netanyahu', 'antisemitism', 'mission', 'bias', 'talks', 'tesla']"
116442,https://www.prnewswire.com/news-releases/whatfix-launches-enterprise-insights-and-ai-features-reimagines-the-future-of-ai-driven-digital-adoption-platforms-dap-301819017.html,2023-05-09,Whatfix Launches Enterprise Insights and AI Features. Reimagines The Future Of AI-Driven Digital Adoption Platforms (DAP),"Topics Business Money Auto TransportationAll Automotive Transportation Aerospace, Defense Air Freight Airlines Aviation Automotive Maritime Shipbuilding Railroads and Intermodal Transportation Sup...",177,Business & Startups,"['adoption', 'info', 'crab', 'makeshift', 'gptbots', 'peers', 'steady', 'shortages', 'manufacturers', 'humanfirst']"
7910,https://www.insidehook.com/daily_brief/tech/25-ai-avatars-lived-small-town,2023-04-12,Here s What Happened When 25 AI Avatars Lived in a Small Town,DesignSex DatingAdviceWatches Travel The 35-Mile Virginia Creeper Trail Is a Secret Foodie Paradise Music The 30 Best Walk-Up Songs in Baseball History Cities Popular at InsideHook ChicagoLos Ange...,53,Business & Startups,"['small', 'entrepreneurs', 'unlocking', 'ango', 'pyramid', 'transformation', 'annotation', 'gramener', 'trainual', 'optimal']"
170167,https://www.pymnts.com/artificial-intelligence-2/2024/mastercard-intros-ai-assistant-for-small-business-owners/,2024-01-11,Mastercard Intros AI Assistant for Small Business Owners,PYMNTS TV Today B2B Retail Fintech Digital Transformation Crypto EMEA Tracker Reports PYMNTS Data Markets More Topics Artifical Intelligence Connected Car Buy Now Pay Later Banking Cloud Cross-Bor...,53,Business & Startups,"['small', 'entrepreneurs', 'unlocking', 'ango', 'pyramid', 'transformation', 'annotation', 'gramener', 'trainual', 'optimal']"


In [ ]:
[print(doc) for doc in sample_data['text_clean'].head(1).values]

Watch LiveVaccineBig Bad B MovieSeen OnContestsCMSD on CW43DealsSee It, Snap It, Send ItNews19 News Investigative UnitUnidentifiedScam SquadCrime400 Years The Vestiges of Slavery in ClevelandCOVID The ClassroomCleveland ComebackDark Side of The LandFirst Alert WeatherRadarClosingsFirst Alert Science SchoolTrafficGas BrownsOperation OrangeTailgate 19Cleveland CavaliersCleveland GuardiansHealthBridging the Great Health DivideVaccineHealth UpdatesGo RedBreast CancerSpot The Signs Opioid CrisisFoodCleveland CooksTaste BudsCommunityShare Your HolidaysCW 43 FocusSee It, Snap It, Send ItCleveland NowCLE WeekendProgramming ScheduleBig Bad B-Movie ShowCribbs in the CLECW43 ClevelandAbout UsMeet the TVGray DC BureauCircle - Country Music LifestyleLatest NewscastsPowerNationPress ReleasesJina AI Shapes Future of Search as CB Insights Names it in 100 Most Innovative AI Startups for Second Year RunningPublished May. 17, 2022 at 12 00 PM EDT Updated 1 hours agoBERLIN, May 17, 2022 PRNewswire -- Lead

[None]

In [ ]:
labels = [gpt_label(doc) for doc in sample_data['text_clean']]
sample_data['sentiment'] = labels

In [ ]:
i=0
df_ai2 = df_ai[~df_ai['url'].isin(sample_data['url'])]
for industry in df_ai2['industry'].unique():
  if industry == 'Technology': sample = df_ai2[df_ai2['industry'] == industry].sample(20)
  else: sample = df_ai2[df_ai2['industry'] == industry].sample(10)
  if i==0:
    sample_data2 = sample
    i=1
  else:
    sample_data2 = pd.concat([sample_data2, sample])
sample_data2.shape

(160, 7)

In [ ]:
sample_data2[~sample_data2['url'].isin(sample_data['url'])].shape

(160, 7)

In [ ]:
labels = [gpt_label(doc) for doc in sample_data2['text_clean']]
sample_data2['sentiment'] = labels

In [ ]:
sample_data2['sentiment'].value_counts()

,count
sentiment,
positive,102
negative,39
neutral,19


In [ ]:
sample_data = pd.concat([sample_data, sample_data2])
sample_data.shape

(320, 8)

In [ ]:
sample_data.head()

,url,date,title_clean,text_clean,topic,industry,Representation,sentiment
53363,https://www.cleveland19.com/prnewswire/2022/05/17/jina-ai-shapes-future-search-cb-insights-names-it-100-most-innovative-ai-startups-second-year-running/,2022-05-17,Jina AI Shapes Future of Search as CB Insights Names it in 100 Most Innovative AI Startups for Second Year Running,"Watch LiveVaccineBig Bad B MovieSeen OnContestsCMSD on CW43DealsSee It, Snap It, Send ItNews19 News Investigative UnitUnidentifiedScam SquadCrime400 Years The Vestiges of Slavery in ClevelandCOVID...",55,Business & Startups,"['list', 'startup', 'lab', 'fero', 'diveplane', 'indian', 'zenerate', 'encora', 'blackbaud', 'invests']",positive
30121,https://www.sakshipost.com/news/musk-confused-how-non-profit-openai-became-30-bn-profit-making-firm-170561,2023-03-16,Musk confused how non-profit OpenAI became 30 bn profit-making firm,News Andhra Pradesh Telangana National International Technology Business Social News Brief Crime Covid 4th Wave Politics Andhra Pradesh Politics Telangana Politics National Politics International ...,10,Business & Startups,"['truthgpt', 'lawsuit', 'profit', 'altman', 'netanyahu', 'antisemitism', 'mission', 'bias', 'talks', 'tesla']",negative
116442,https://www.prnewswire.com/news-releases/whatfix-launches-enterprise-insights-and-ai-features-reimagines-the-future-of-ai-driven-digital-adoption-platforms-dap-301819017.html,2023-05-09,Whatfix Launches Enterprise Insights and AI Features. Reimagines The Future Of AI-Driven Digital Adoption Platforms (DAP),"Topics Business Money Auto TransportationAll Automotive Transportation Aerospace, Defense Air Freight Airlines Aviation Automotive Maritime Shipbuilding Railroads and Intermodal Transportation Sup...",177,Business & Startups,"['adoption', 'info', 'crab', 'makeshift', 'gptbots', 'peers', 'steady', 'shortages', 'manufacturers', 'humanfirst']",neutral
7910,https://www.insidehook.com/daily_brief/tech/25-ai-avatars-lived-small-town,2023-04-12,Here s What Happened When 25 AI Avatars Lived in a Small Town,DesignSex DatingAdviceWatches Travel The 35-Mile Virginia Creeper Trail Is a Secret Foodie Paradise Music The 30 Best Walk-Up Songs in Baseball History Cities Popular at InsideHook ChicagoLos Ange...,53,Business & Startups,"['small', 'entrepreneurs', 'unlocking', 'ango', 'pyramid', 'transformation', 'annotation', 'gramener', 'trainual', 'optimal']",positive
170167,https://www.pymnts.com/artificial-intelligence-2/2024/mastercard-intros-ai-assistant-for-small-business-owners/,2024-01-11,Mastercard Intros AI Assistant for Small Business Owners,PYMNTS TV Today B2B Retail Fintech Digital Transformation Crypto EMEA Tracker Reports PYMNTS Data Markets More Topics Artifical Intelligence Connected Car Buy Now Pay Later Banking Cloud Cross-Bor...,53,Business & Startups,"['small', 'entrepreneurs', 'unlocking', 'ango', 'pyramid', 'transformation', 'annotation', 'gramener', 'trainual', 'optimal']",positive


In [ ]:
sample_data['sentiment'].value_counts()

,count
sentiment,
positive,212
negative,69
neutral,39


In [ ]:
pd.set_option('display.max_colwidth', 200)
sample_data[sample_data['sentiment'] == 'positive'].sample(10)

,url,date,title_clean,text_clean,topic,industry,Representation,sentiment
79224,https://www.mysuncoast.com/prnewswire/2023/06/02/asco-23-ai-precision-oncology-company-perthera-unveils-premium-experience-clinicians-with-new-options-ordering-access-enhanced-perthera-analytics/,2023-06-03,"At ASCO '23, AI Precision Oncology Company Perthera Unveils Premium Experience for Clinicians with New Options for Ordering and Access to Enhanced Perthera Analytics",ContestsNewsAs Seen On 7Saluting the SuncoastLocal NewsWatch LiveCrimeHealthRed StrongSportsTrafficGas PricesShare It!Submit a News TreeAnimal OuttakesCommunity CalendarChalkboard ChampionsDiscove...,155,Food & Restaurants,"['drive', 'ordering', 'mcdonald', 'thrus', 'perthera', 'sneaker', 'circular', 'test', 'asco', 'wenspok']",positive
103641,https://www.nasdaq.com/articles/predicting-the-7-top-performing-stocks-for-2024-with-ai-technology,2023-11-09,Predicting the 7 Top Performing Stocks for 2024 With AI Technology,Predicting the 7 Top Performing Stocks for 2024 With AI Technology Nasdaq Skip to main content Nasdaq Weekly Macro Scorecard Market Activity Market Activity - Stocks Options ETFs Mutual Funds Inde...,3,Finance & Stock Markets,"['stocks', 'wall', 'bubble', 'bull', 'cathie', 'consider', 'dividend', 'growth', 'rally', 'penny']",positive
65709,https://www.dakotanewsnow.com/prnewswire/2023/05/04/ets-receives-nearly-325-million-ai-research-institute-advance-stem-learning-among-marginalized-us-students/,2023-05-04,ETS Receives Nearly 3.25 Million for AI Research Institute To Advance STEM Learning Among Marginalized U.S. Students,A RadarSioux FallsNortheast SDCentral SDDelays and CancellationsSkycam NetworkMap RoomRoad Of The WeekPlay of the WeekPigskin PreviewPoliticsElection ResultsNational Election MapsSouth Dakota Elec...,9,Education,"['higher', 'teaching', 'universities', 'adept', 'lie', 'district', 'stem', 'hybriu', 'aureus', 'shootings']",positive
134913,https://www.forbes.com/sites/elijahclark/2023/08/25/exploring-the-benefits-of-explainable-artificial-intelligence-xai-in-the-realm-of-marketing/,2023-08-25,Exploring The Benefits Of EXplainable Artificial Intelligence (XAI) In The Realm Of Marketing,"ForbesAug 25, 2023,07 00am EDTFlipping The Script On AI By Using Prompt Engineering Techniques To Have Generative AI Ask You Questions And Get Even Better Answers Out Of The AIAug 24, 2023,02 41pm...",16,Marketing & Sales,"['advertising', 'retail', 'experiences', 'revolutionizing', 'brandequity', 'influencer', 'fit', 'digital', 'infor', 'user']",positive
72955,https://www.motionsonline.org/worldwide/cloud-machine-learning-operations-mlops-market-2024-trends-ibm-datarobot-sas/77514/,2024-05-06,"Cloud Machine Learning Operations (MLOps) Market 2024 Trends - IBM, DataRobot, SAS",".com. The report on the Cloud Machine Learning Operations (MLOps) Market provides valuable insights into the global industry, offering essential information and statistical data. This study thorou...",19,Technology,"['partner', 'docs', 'softwarereviews', 'snowflake', 'vertex', 'cognizant', 'cloudera', 'accelerate', 'retailers', 'migrate']",positive
39697,https://www.kvcrnews.org/2024-01-18/how-should-judges-and-lawyers-use-artificial-intelligence,2024-01-18,How should judges and lawyers use artificial intelligence?,"2024 91.9 KVCRKVCR is a service of the San Bernardino Community College District. San Bernardino Community College District does not discriminate on the basis of age, color, creed, religion, disab...",33,Legal Services,"['tax', 'everlaw', 'litigation', 'schumacher', 'willingness', 'family', 'teams', 'drafting', 'interview', 'evolving']",positive
90155,https://www.wionews.com/videos/south-korean-startup-launches-ai-chips-561954,2023-02-14,South Korean startup launches AI Chips,South Korean startup launches AI Chips - World NewsvideosGravitasWorldMY CommunityLIVE TVWorldCentral AsiaWest AsiaEast AsiaSouth East and New ZealandAfricaSouth TvBusiness Summit Dubai 2022Global...,212,Government & 

In [ ]:
sample_data.to_csv('labeled_sample_2types.csv', index=False)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sentence_transformers import SentenceTransformer

In [ ]:
X = sample_data["title_clean"].astype(str).tolist()
y = np.where(sample_data["sentiment"]=='neutral', 'negative', 'positive')

In [ ]:
model = SentenceTransformer("distilbert-base-nli-mean-tokens")
X_emb = model.encode(X, show_progress_bar=True)

Batches:   0%|          | 0/10 [00:00<?, ?it/s]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_emb, y, test_size=0.2, random_state=42, stratify=y)
clf = LogisticRegression(max_iter=1000,class_weight='balanced')
clf.fit(X_train, y_train)

LogisticRegression(class_weight='balanced', max_iter=1000)

In [ ]:
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

    negative       0.09      0.12      0.11         8
    positive       0.87      0.82      0.84        56

    accuracy                           0.73        64
   macro avg       0.48      0.47      0.47        64
weighted avg       0.77      0.73      0.75        64



In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[ 1,  7],
       [10, 46]])

## Fine-tune transformer

In [ ]:
label_map = {'positive': 1, 'neutral': 0, 'negative': 0}
sample_data['label'] = sample_data['sentiment'].map(label_map)

In [ ]:
from transformers import AutoTokenizer

model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(examples):
    return tokenizer(examples['text_clean'], padding="max_length", truncation=True)

from datasets import Dataset

dataset = Dataset.from_pandas(sample_data[['text_clean', 'label']])
tokenized_dataset = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/320 [00:00<?, ? examples/s]

In [ ]:
tokenized_dataset = tokenized_dataset.train_test_split(test_size=0.2)

In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
import numpy as np
from sklearn.metrics import accuracy_score, f1_score

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1": f1_score(labels, preds)
    }

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_steps=10,
    save_total_limit=1
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    compute_metrics=compute_metrics
)

trainer.train()

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.500800,0.502070,0.765625,0.854369
2,0.481700,0.455296,0.781250,0.847826
3,0.330300,0.382392,0.812500,0.866667
4,0.242900,0.371440,0.812500,0.863636
5,0.215300,0.366646,0.812500,0.869565


TrainOutput(global_step=160, training_loss=0.3820883795619011, metrics={'train_runtime': 17.6626, 'train_samples_per_second': 72.469, 'train_steps_per_second': 9.059, 'total_flos': 169558270279680.0, 'train_loss': 0.3820883795619011, 'epoch': 5.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.366645872592926,
 'eval_accuracy': 0.8125,
 'eval_f1': 0.8695652173913043,
 'eval_runtime': 0.3158,
 'eval_samples_per_second': 202.677,
 'eval_steps_per_second': 25.335,
 'epoch': 5.0}

In [ ]:
outputs = trainer.predict(tokenized_dataset["test"])
preds = np.argmax(outputs.predictions, axis=1)
labels = outputs.label_ids
target_names = ["negative", "positive"]

print(classification_report(labels, preds, target_names=target_names))

              precision    recall  f1-score   support

    negative       0.71      0.63      0.67        19
    positive       0.85      0.89      0.87        45

    accuracy                           0.81        64
   macro avg       0.78      0.76      0.77        64
weighted avg       0.81      0.81      0.81        64



In [ ]:
import torch
from tqdm import tqdm

def predict_sentiment(texts, model, tokenizer, batch_size=32, device="cpu"):
    model.eval()
    model.to(device)
    predictions = []
    confidences = []

    for i in tqdm(range(0, len(texts), batch_size)):
        batch_texts = texts[i:i+batch_size]
        inputs = tokenizer(batch_texts, truncation=True, padding=True, max_length=512, return_tensors="pt").to(device)

        with torch.no_grad():
            outputs = model(**inputs)
            probs = torch.nn.functional.softmax(outputs.logits, dim=-1)

            preds = torch.argmax(probs, axis=1).cpu().tolist()
            confs = probs[:, 1].cpu().tolist()

            predictions.extend(preds)
            confidences.extend(confs)

    return predictions, confidences

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using:", device)

Using: cuda


In [ ]:
key_words = ['premium png']
print(df_ai.shape)
pattern = '|'.join(map(re.escape, key_words))
df_ai[df_ai['title_clean'].str.contains(pattern, case=False, na=False)].shape
df_ai = df_ai[~df_ai['title_clean'].str.contains(pattern, case=False, na=False)]
df_ai = df_ai[df_ai['title_clean'] != '']
df_ai.dropna(subset=['title_clean'], inplace=True)
print(df_ai.shape)

(181339, 7)
(181300, 7)


In [ ]:
texts = df_ai['text_clean'].astype(str).fillna("").tolist()

In [ ]:
preds, probs = predict_sentiment(texts, model, tokenizer, device="cuda")
df_ai["sentiment"] = preds
df_ai["sentiment_score"] = probs

100%|██████████| 5666/5666 [12:26<00:00,  7.59it/s]


In [ ]:
len(preds)

181299

In [ ]:
df_ai['sentiment'].value_counts()

,count
sentiment,
1,133261
0,48038


In [ ]:
df_ai['sentiment'] = np.where(df_ai['sentiment']==0, 'negative', 'positive')

In [ ]:
df_ai['sentiment'].value_counts()

,count
sentiment,
positive,133261
negative,48038


In [ ]:
df_ai.to_csv('labeled_ai_data.csv', index=False)

In [ ]:
df_ai = pd.read_csv('labeled_ai_data.csv')

In [ ]:
df_ai[df_ai['sentiment'] == 'positive'].sample(10)

,url,date,title_clean,text_clean,topic,industry,Representation,sentiment,sentiment_score
61408,https://www.thehindu.com/sci-tech/technology/businesses-tech-groups-warn-eu-against-over-regulating-ai-foundation-models/article67568625.ece,2023-11-24,"Businesses, tech groups warn EU against over-regulating AI foundation models","Sci-Tech Technology Businesses, tech groups warn EU against over-regulating AI foundation models Businesses and tech groups on Thursday warned the European Union against over-regulating artificial...",20,Government & Policy,"['europe', 'regulation', 'agreed', 'affect', 'guardrails', 'draft', 'conduct', 'tougher', 'xtremeai', 'woflow']",positive,0.855042
77408,https://www.thewestonforum.com/research-abb-opens-innovation-campus-for-automation-and-artificial-intelligence-digital-trends-science-and-research-news-todays-logistics/,2022-07-13,"Research ABB Opens Innovation Campus for Automation and Artificial Intelligence - Digital, Trends, Science and Research - news","news Today s Logistics July 13, 2022 Faye Stephens Technology company ABB has opened a new global campus for innovation and education at the headquarters of its subsidiary B R in Eggelsberg (Austr...",185,Technology,"['groundbreaking', 'shatter', 'semiconductors', 'athletes', 'greener', 'birdsongs', 'rebranding', 'asig', 'profits', 'radimagenet']",positive,0.961035
13458,https://www.wcax.com/prnewswire/2023/06/28/rezilion-report-finds-worlds-most-popular-generative-ai-projects-present-high-security-risk/,2023-06-28,Rezilion Report Finds World's Most Popular Generative AI Projects Present High Security Risk,Order Channel 3 News StoriesCommunity PartnersAdvertise With UsWork for WCAXPaid CalendarHike of the WeekMarketplaceWYCIAcross the FenceBuy Vermont FirstCircle - Country Music BreakPress ReleasesR...,81,Technology,"['blinken', 'bigai', 'hiddenlayer', 'llms', 'summits', 'mellon', 'vsblty', 'classification', 'threats', 'center']",positive,0.965962
25007,https://www.etftrends.com/energy-infrastructure-channel/midstream-mlps-ai-adds-positive-natural-gas-outlook/,2024-04-30,Midstream MLPs AI Adds to Positive Natural Gas Outlook,Midstream MLPs AI Adds to Positive Natural Gas Outlook Expert InsightsChannels Themes Active ETFsETF InvestingAlternativesETF StrategistArtificial IntelligenceETF YieldBeyond Basic BetaFinancial L...,51,Energy & Environment,"['gas', 'whitepaper', 'renewable', 'transition', 'shipping', 'drink', 'vertiv', 'iccsd', 'reputation', 'demand']",positive,0.962900
140406,https://cio.economictimes.indiatimes.com/amp/news/artificial-intelligence/cadence-to-sell-ai-supercomputer-for-jet-design-software/107343403,2024-02-02,"Cadence to sell AI supercomputer for jet design software, CIO News, ET CIO","Artificial Intelligence 2 min read Cadence to sell AI supercomputer for jet design software Cadence is best know for its software that helps design computer chips, where the precise placement of t...",278,Technology,"['bangdb', 'supercomputers', 'fuel', 'training', 'alps', 'cyberport', 'sets', 'denmark', 'cadence', 'centre']",positive,0.967614
66397,https://www.foxbangor.com/news/national/meet-the-72-year-old-congressman-going-back-to-college-to-learn-about-ai/article_e11e23d1-0e95-5e8a-bb86-01aeee2643f7.html,2023-04-24,Meet the 72-year-old congressman going back to college to learn about AI - National,News Local News National Newscast Community Calendar Bios Sports Local Sports National Sports Features Health Live Interviews Watch ABC Weekend Warriors WWII Stories Weather Local Weather School B...,191,Government & Policy,"['senators', 'regulation', 'tackle', 'civilized', 'blackout', 'forum', 'zuckerberg', 'pass', 'door', 'tech']",positive,0.552761
159752,https://www.14news.com/prnewswire/2023/08/04/huaweis-pangu-weather-ai-model-can-predict-weather-events-seconds-just-released-public-free/,2023-08-04,Huawei's Pangu-Weather AI model can predict weather events in seconds just released to the public for free,castsGray DC - Country Music Lifestyl

In [ ]:
df_ai[df_ai['sentiment'] == 'negative'].sample(10)

,url,date,title_clean,text_clean,topic,industry,Representation,sentiment,sentiment_score
31719,https://www.dailymail.co.uk/news/article-12143821/Recording-exec-warns-Republican-issues-stemming-rise-AI-fake-music.html,2023-05-31,Recording exec warns top Republican on issues stemming from rise of AI and 'fake music',"Feedback Wednesday, May 31st 2023 10AM 70 F 1PM 78 F 5-Day Forecast Advertisement Snitches get stitches! Ghislaine Maxwell is living in 'constant fear' of a brutal revenge attack by two violent Cu...",50,Technology,"['elections', 'deepfakes', 'guardrails', 'fade', 'disinformation', 'polluted', 'posing', 'media', 'interference', 'worry']",negative,0.109981
119888,https://www.newswire.ca/news-releases/frost-amp-sullivan-recognizes-infosys-for-accelerating-digital-transformation-addressing-real-world-problems-and-maximizing-value-for-clients-with-its-applied...,2022-06-15,"Frost Sullivan Recognizes Infosys for Accelerating Digital Transformation, Addressing Real-world Problems, and Maximizing Value for Clients with Its Applied AI Offering","Topics Business Auto TransportationAll Automotive Transportation Aerospace, Defense Air Freight Airlines Aviation Automotive Maritime Shipbuilding Railroads and Intermodal Transportation Supply Ch...",99,Technology,"['servicenow', 'databricks', 'eliminating', 'delays', 'teleperformance', 'topaz', 'seecares', 'northmarq', 'lakehouse', 'domino']",negative,0.068222
119968,https://thewest.com.au/politics/nines-sexist-ai-image-of-mp-labelled-a-wake-up-call-c-13415941,2024-01-31,Nine's 'sexist' AI image of MP labelled a wake-up call,r ExclusiveLatestToday's PaperPuzzlesTimespoolWest RewardsVanishing CousinsUp LateThe West LiveCourt in the ActNewsReveal sub navigationChevron Down IconCoronavirusWA sub navigationChevron Down Ic...,77,Technology,"['generator', 'explicit', 'hole', 'makeover', 'headshots', 'children', 'katy', 'met', 'black', 'catches']",negative,0.161978
121278,https://www.fox9.com/news/entertainment-companies-hiring-ai-specialists-amid-hollywood-strike,2023-07-30,Entertainment companies hiring AI specialists amid Hollywood strike,site News Local NewsNational NewsWorld For ChangeFOX News SundayWeather FOX 9 Weather AppForecastSchool ClosingsLive Weather CamerasTrafficFOX WeatherSports Shayne WellsGarden GuyRecipesMoney Pers...,148,Media & Entertainment,"['strike', 'copyrights', 'patterson', 'obscenity', 'sag', 'thousands', 'aftra', 'oscars', 'doubts', 'netflix']",negative,0.329254
93886,https://www.wwlp.com/news/political-news/ap-politics/ap-amazon-google-meta-microsoft-and-other-tech-firms-agree-to-ai-safeguards-set-by-the-white-house/,2023-07-21,"Amazon, Google, Meta, Microsoft and other tech firms agree to AI safeguards set by the White House",Watch Now Watch WWLP Newscast Watch Mass Appeal Newscasts on CW Springfield Video Center TV Schedule What s On NBC Children s Programming News Local News 22News I-Team Massachusetts News Breaking ...,251,Government & Policy,"['agree', 'house', 'seoul', 'vetoes', 'consortium', 'risks', 'unifi', 'amazon', 'california', 'science']",negative,0.079776
83252,https://www.the-express.com/entertainment/celebrity-news/131263/Taylor-Swift-fans-called-save-America-AI-deepfakes,2024-03-15,Taylor Swift fans called in to save America as government issues warning over AI fakes - Celebrity News - Entertainment,EntertainmentCelebrity News Taylor Swift fans called in to save America as government issues warning over AI fakesUS politician pleads for Swifties to save America' as she issued a bleak warning o...,143,Media & Entertainment,"['swift', 'williams', 'daughter', 'recreations', 'inflation', 'viral', 'trump', 'falsely', 'deepfake', 'sxsw']",negative,0.270424
170337,https://www.wuwm.com/health-science/health-science/2023-01-19/therapy-by-chatbot-the-promise-and-challenges-in-using-ai-for-mental-health,2023-01-19,Therapy by chatbot? The promise and challenges in using AI for mental health - WUWM 89.7 FM,"ed her daughter and mother by baking recipes sh

In [ ]:
# drop some index

df_ai.drop(index=[144760], inplace=True)

In [ ]:
df_ai.shape

(181299, 8)

In [ ]:
df_ai.groupby('industry')['sentiment_score'].mean()

,sentiment_score
industry,
Automotive,0.726147
Business & Startups,0.771899
Cybersecurity,0.726120
Education,0.701317
Employment & Workforce,0.625504
Energy & Environment,0.769401
Finance & Stock Markets,0.660842
Food & Restaurants,0.809881
Government & Policy,0.687482


In [ ]:
df_ai.groupby('industry')['sentiment'].value_counts(normalize=True)

industry                 sentiment
Automotive               positive     0.758947
                         negative     0.241053
Business & Startups      positive     0.798250
                         negative     0.201750
Cybersecurity            positive     0.758963
                         negative     0.241037
Education                positive     0.730559
                         negative     0.269441
Employment & Workforce   positive     0.622716
                         negative     0.377284
Energy & Environment     positive     0.802478
                         negative     0.197522
Finance & Stock Markets  positive     0.709103
                         negative     0.290897
Food & Restaurants       positive     0.851064
                         negative     0.148936
Government & Policy      positive     0.711086
                         negative     0.288914
Healthcare               positive     0.820017
                         negative     0.179983
Legal Services           positive     0.656532
                         negative     0.343468
Lifestyle & Family       positive     0.764256
                         negative     0.235744
Marketing & Sales        positive     0.874870
                         negative     0.125130
Media & Entertainment    positive     0.543142
                         negative     0.456858
Technology               positive     0.751455
                         negative     0.248545
Name: proportion, dtype: float64

In [ ]:
df_ai.to_csv('/content/drive/MyDrive/UChicago/NLP/labeled_ai_data.csv', index=False)

In [ ]:
sample_data.to_csv('/content/drive/MyDrive/UChicago/NLP/labeled_sample.csv', index=False)

In [ ]:
top10_2 = pd.read_csv('/content/drive/MyDrive/UChicago/NLP/topic_info_2.csv')

In [ ]:
top10_2.loc[top10_2['Topic'] == 50,['Topic','Representation','Representative_Docs','industry']]

,Topic,Representation,Representative_Docs,industry
51,50,"['elections', 'deepfakes', 'guardrails', 'fade', 'disinformation', 'polluted', 'posing', 'media', 'interference', 'worry']","['How real is the threat of deepfakes in the 2024 election?', 'How real is the threat of deepfakes in the 2024 election?', 'How real is the threat of deepfakes in the 2024 election?']",Technology


In [ ]:
df_ai[df_ai['topic']==50].sample(10)

,url,date,title_clean,text_clean,topic,industry,Representation,sentiment,sentiment_score
15291,https://www.kshb.com/science-and-tech/artificial-intelligence/wyoming-man-tries-making-ai-mayor-of-cheyenne,2024-06-19,Wyoming man tries making AI mayor of Cheyenne,Weather 10 Day Forecast Hourly Forecast Weather Cams Weather Radar Storm Tracker Weather Alerts School Closings and Delays Weather Blog School Day at the K Traffic Gas Prices News Local News Video...,50,Technology,"['elections', 'deepfakes', 'guardrails', 'fade', 'disinformation', 'polluted', 'posing', 'media', 'interference', 'worry']",negative,0.288557
120085,https://www.smh.com.au/technology/government-urged-to-ban-ai-generated-election-materials-20240521-p5jfas.html,2024-05-21,"Election 2025 AI ban urged by David Pocock, Kate Chaney",SydneyNSWPoliticsShow AustraliaBusinessShow N' BearsThe economyBanking subsectionsNorth AmericaEuropeAsiaMiddle EastOceaniaSouth AmericaAfricaNationalShow subsectionsNewsDomain MagazineTitle subse...,50,Technology,"['elections', 'deepfakes', 'guardrails', 'fade', 'disinformation', 'polluted', 'posing', 'media', 'interference', 'worry']",positive,0.500218
54677,https://www.thewrap.com/jake-tapper-cnn-deepfake-ai-misleading-ads-amy-klobuchar/,2024-10-05,Jake Tapper Uses Deepfake of Himself to Warn of AI Election Interference,TheWrapBook TheWrapBook The Art of Television The WrapBook The Art of Cinema WrapStyle TV TV Ratings Reviews Movies Movies Steve Pond Reviews Box Office Sundance Sundance Videos TIFF Report From T...,50,Technology,"['elections', 'deepfakes', 'guardrails', 'fade', 'disinformation', 'polluted', 'posing', 'media', 'interference', 'worry']",negative,0.137453
41198,https://www.mrt.com/news/article/how-ai-could-take-over-elections-and-18131814.php,2023-06-02,How AI could take over elections,r Servicese-EditionPrivacy NoticeTerms of and Honor RollSportsHigh School Sports2022 HS Football EnergyBusiness DirectoryOil EnergyFYIHealthy LivingBusiness InsiderReaders' ChoiceAround the Oil Pa...,50,Technology,"['elections', 'deepfakes', 'guardrails', 'fade', 'disinformation', 'polluted', 'posing', 'media', 'interference', 'worry']",positive,0.588415
24801,https://news.yahoo.com/legislature-eyes-ai-issues-2024-205018100.html,2023-12-24,Legislature eyes AI issues in 2024,tickers Futures World indices US Treasury bonds Currencies Crypto Top ETFs Top mutual funds Highest open interest Highest implied volatility Currency converter Sectors Sectors Basic materials Comm...,50,Technology,"['elections', 'deepfakes', 'guardrails', 'fade', 'disinformation', 'polluted', 'posing', 'media', 'interference', 'worry']",negative,0.068569
109879,https://abc7chicago.com/artificial-intelligence-misinformation-disinformation-ai/14308115/,2024-01-11,"Artificial intelligence AI-powered misinformation is the world's biggest short-term threat, Davos report says","Artificial intelligence AI-powered misinformation is the world's biggest short-term threat, Davos report says - ABC7 Chicago WEATHER ALERTWinter Storm WatchFull StoryWATCH LIVEChicago Suburban Coo...",50,Technology,"['elections', 'deepfakes', 'guardrails', 'fade', 'disinformation', 'polluted', 'posing', 'media', 'interference', 'worry']",positive,0.932753
77909,https://www.hindustantimes.com/world-news/what-if-weakest-us-president-is-re-elected-ai-ad-fuels-fake-news-concerns-101682504974664-amp.html,2023-04-26,"What if weakest US President is re-elected , AI ad fuels fake news concerns - World News","World News What if weakest US President is re-elected , AI ad fuels fake news concerns What if weakest US President is re-elected , AI ad fuels fake news concerns ByHT Team Apr 26, 2023 07 32 PM I...",50,Technology,"['elections', 'deepfakes', 'guardrails', 'fade', 'disinformation', 'polluted', 'posing', 'media', 'interference', 'worry']",negative,0.239189
44350,https://www.wjtv.com/news/national/ap-as-social-media-guardrails-fade-and-ai-deepfakes-go-mainstream-experts-warn-of-impact-on-elections/,2023-12-2

In [ ]:
# y_pred = clf.predict(X_test)
# print(classification_report(y_test, y_pred))

# # 5. Predict all data (for example, on df_ai)
# df_all = pd.read_csv("all_articles.csv")  # Ensure 'title_clean' column exists
# X_all = df_all["title_clean"].astype(str).tolist()
# X_all_emb = model.encode(X_all, show_progress_bar=True)
# df_all["predicted_sentiment"] = clf.predict(X_all_emb)

# # Save predictions
# df_all.to_csv("ai_sentiment_predictions.csv", index=False)